In [27]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [28]:
import torch
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.stats import norm
from pyDOE import lhs

import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import akshare as ak
import yfinance as yf
import pandas as pd
from arch import arch_model

import warnings
warnings.filterwarnings('ignore')

np.random.seed(1234)

# CUDA support
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [29]:
# 神经网络模型
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        self.depth = len(layers) - 1
        self.activation = torch.nn.Tanh
        
        layer_list = []
        for i in range(self.depth - 1):
            layer_list.append(('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1])))
            layer_list.append(('activation_%d' % i, self.activation()))
        
        layer_list.append(('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1])))
        layerDict = OrderedDict(layer_list)
        self.layers = torch.nn.Sequential(layerDict)
    
    def forward(self, x):
        out = self.layers(x)
        return out


In [30]:
# 物理信息神经网络
class PhysicsInformedNN():
    def __init__(self, X, u, layers, lb, ub, r):
        # 归一化参数
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # 数据
        self.S = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        
        # 模型参数
        self.r = r
        
        # 可学习参数
        self.lambda_2 = torch.tensor([0.5], requires_grad=True).to(device)  # σ
        self.lambda_3 = torch.tensor([0.0], requires_grad=True).to(device)  # β
        
        self.lambda_2 = torch.nn.Parameter(self.lambda_2)
        self.lambda_3 = torch.nn.Parameter(self.lambda_3)
        
        # 神经网络
        self.dnn = DNN(layers).to(device)
        self.dnn.register_parameter('lambda_2', self.lambda_2)
        self.dnn.register_parameter('lambda_3', self.lambda_3)
        
        # 优化器
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(),
            lr=1.0,
            max_iter=50000,
            max_eval=50000,
            history_size=50,
            tolerance_grad=1e-5,
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"
        )
        
        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters())
        self.iter = 0
    
    def net_u(self, S, t):
        u = self.dnn(torch.cat([S, t], dim=1))
        return u
    
    def net_f(self, S, t):
        # 转换回原始S值
        S_original = self.lb[0] + S * (self.ub[0] - self.lb[0])
        
        # 可学习参数
        sigma = torch.exp(self.lambda_2)
        beta = torch.exp(self.lambda_3)
        
        # 微分计算
        u = self.net_u(S, t)
        
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        u_S = torch.autograd.grad(u, S, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        u_SS = torch.autograd.grad(u_S, S, grad_outputs=torch.ones_like(u_S), retain_graph=True, create_graph=True)[0]
        
        # CEV模型PDE
        diffusion = 0.5 * sigma**2 * (S_original)**(2*beta) * u_SS
        convection = self.r * S_original * u_S
        reaction = -self.r * u
        f = u_t + diffusion + convection + reaction
        
        return f
    
    def loss_func(self):
        u_pred = self.net_u(self.S, self.t)
        f_pred = self.net_f(self.S, self.t)
        loss = torch.mean((self.u - u_pred)**2) + torch.mean(f_pred**2)
        
        self.optimizer.zero_grad()
        loss.backward()
        
        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Loss: %e, Sigma: %.5f, Beta: %.5f' %
                (
                    loss.item(),
                    torch.exp(self.lambda_2.detach()).item(),
                    torch.exp(self.lambda_3.detach()).item()
                )
            )
        return loss
    
    def train(self, nIter):
        self.dnn.train()
        for epoch in range(nIter):
            u_pred = self.net_u(self.S, self.t)
            f_pred = self.net_f(self.S, self.t)
            loss = torch.mean((self.u - u_pred)**2) + torch.mean(f_pred**2)
            
            # Adam优化
            self.optimizer_Adam.zero_grad()
            loss.backward()
            self.optimizer_Adam.step()
            
            if epoch % 100 == 0:
                print(
                    'It: %d, Loss: %.3e, Sigma: %.6f, Beta: %.6f' %
                    (
                        epoch,
                        loss.item(),
                        torch.exp(self.lambda_2).item(),
                        torch.exp(self.lambda_3).item()
                    )
                )
        
        # L-BFGS优化
        self.optimizer.step(self.loss_func)
    
    def predict(self, X):
        S = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        
        self.dnn.eval()
        u = self.net_u(S, t)
        f = self.net_f(S, t)
        return u.detach().cpu().numpy(), f.detach().cpu().numpy()

In [31]:
# 数据加载与处理
option_sse_daily_sina_df = ak.option_sse_daily_sina(symbol='10006687')
option_sse_daily_sina_df = option_sse_daily_sina_df.drop_duplicates()
option_sse_daily_sina_df['日期'] = pd.to_datetime(option_sse_daily_sina_df['日期']).dt.strftime('%Y-%m-%d')

df = ak.fund_etf_hist_sina(symbol="sh510300")
df["date"] = pd.to_datetime(df["date"])
start_date = "2024-01-18"
end_date = "2024-06-26"
mask = (df["date"] >= start_date) & (df["date"] <= end_date)
filtered_df = df.loc[mask].reset_index(drop=True)
filtered_df['date'] = filtered_df['date'].dt.strftime('%Y-%m-%d')
hist = filtered_df[['date', 'close']]
hist.columns = ['日期', '股票收盘价']

data = pd.merge(option_sse_daily_sina_df, hist, on='日期', how='inner')
data = data.sort_values('日期').reset_index(drop=True)
train_data = data[['日期', '收盘', '股票收盘价']]

# 数据预处理
train_data['日期'] = pd.to_datetime(train_data['日期']).astype(np.int64) // 10**9
u_market = train_data['收盘'].values
t_market = train_data['日期'].values
S_market = train_data['股票收盘价'].values

X_star = np.hstack((S_market[:, None], t_market[:, None]))
u_star = u_market[:, None]

# 归一化
lb = X_star.min(0)
ub = X_star.max(0)
X_train = (X_star - lb) / (ub - lb)
u_train = (u_star - u_star.min()) / (u_star.max() - u_star.min())

# 模型参数
r = 0.0141  # 无风险利率

In [32]:
# 训练模型
layers = [2, 30, 30, 30, 30, 30, 30, 30, 1]
model = PhysicsInformedNN(X_train, u_train, layers, lb, ub, r)
model.train(10000)

# 结果评估
u_pred, f_pred = model.predict(X_star)
error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)

lambda_2_value = np.exp(model.lambda_2.detach().cpu().numpy())
lambda_3_value = np.exp(model.lambda_3.detach().cpu().numpy())

print('Estimated Sigma:', lambda_2_value)
print('Estimated Beta:', lambda_3_value)
print('Relative L2 Error:', error_u)

It: 0, Loss: 4.549e-01, Sigma: 1.647073, Beta: 0.999001
It: 100, Loss: 1.821e-02, Sigma: 1.587647, Beta: 0.962107
It: 200, Loss: 1.472e-02, Sigma: 1.536353, Beta: 0.931145
It: 300, Loss: 9.239e-03, Sigma: 1.490820, Beta: 0.903849
It: 400, Loss: 5.198e-03, Sigma: 1.455147, Beta: 0.882630
It: 500, Loss: 2.814e-03, Sigma: 1.423366, Beta: 0.864066
It: 600, Loss: 1.840e-03, Sigma: 1.395794, Beta: 0.848232
It: 700, Loss: 1.468e-03, Sigma: 1.373976, Beta: 0.835903
It: 800, Loss: 1.253e-03, Sigma: 1.365748, Beta: 0.831356
It: 900, Loss: 1.137e-03, Sigma: 1.350845, Beta: 0.823086
It: 1000, Loss: 1.047e-03, Sigma: 1.345039, Beta: 0.819930
It: 1100, Loss: 9.777e-04, Sigma: 1.330476, Beta: 0.811929
It: 1200, Loss: 9.142e-04, Sigma: 1.326099, Beta: 0.809584
It: 1300, Loss: 8.732e-04, Sigma: 1.312419, Beta: 0.802137
It: 1400, Loss: 8.181e-04, Sigma: 1.308968, Beta: 0.800315
It: 1500, Loss: 8.115e-04, Sigma: 1.295722, Beta: 0.793162
It: 1600, Loss: 7.427e-04, Sigma: 1.292868, Beta: 0.791675
It: 1700,